<a href="https://colab.research.google.com/github/Amala02/EmoTer/blob/main/emoTer_ver7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers nltk 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 102.4 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import huggingface_hub as hb

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import tensorflow as tf 
if tf.config.list_physical_devices('GPU'):
    physical_devices = tf.config.list_physical_devices('GPU')
    tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)
    tf.config.experimental.set_virtual_device_configuration(physical_devices[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=5000)])
physical_devices,tf.__version__

Mounted at /content/drive


([PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')], '2.9.2')

In [ ]:
# importing the dataset 
df_train = pd.read_csv('/content/drive/MyDrive/dataset/train.txt', header =None, sep =';', names = ['Input','Sentiment'], encoding='utf-8')
df_test = pd.read_csv('/content/drive/MyDrive/dataset/test.txt', header = None, sep =';', names = ['Input','Sentiment'],encoding='utf-8')
df_val=pd.read_csv('/content/drive/MyDrive/dataset/val.txt',header=None,sep=';',names=['Input','Sentiment'],encoding='utf-8')

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from nltk.corpus import stopwords
stop_words = stopwords.words("english")
import re
import string

In [ ]:
df_train.head()

,Input,Sentiment
0,i didnt feel humiliated,sadness
1,i can go from feeling so hopeless to so damned...,sadness
2,im grabbing a minute to post i feel greedy wrong,anger
3,i am ever feeling nostalgic about the fireplac...,love
4,i am feeling grouchy,anger


In [ ]:
df_train["Sentiment"].unique()

array(['sadness', 'anger', 'love', 'surprise', 'fear', 'joy'],
      dtype=object)

In [ ]:
df_full = pd.concat([df_train,df_test,df_val], axis = 0)
df_full

,Input,Sentiment
0,i didnt feel humiliated,sadness
1,i can go from feeling so hopeless to so damned...,sadness
2,im grabbing a minute to post i feel greedy wrong,anger
3,i am ever feeling nostalgic about the fireplac...,love
4,i am feeling grouchy,anger
...,...,...
1995,im having ssa examination tomorrow in the morn...,sadness
1996,i constantly worry about their fight against n...,joy
1997,i feel its important to share this info for th...,joy
1998,i truly feel that if you are passionate enough...,joy


In [ ]:
!wget -P data/full_dataset/ https://storage.googleapis.com/gresearch/goemotions/data/full_dataset/goemotions_1.csv

--2023-01-23 02:55:31--  https://storage.googleapis.com/gresearch/goemotions/data/full_dataset/goemotions_1.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.12.128, 172.217.194.128, 74.125.200.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.12.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14174600 (14M) [application/octet-stream]
Saving to: ‘data/full_dataset/goemotions_1.csv’

goemotions_1.csv    100%[===================>]  13.52M  9.30MB/s    in 1.5s    

2023-01-23 02:55:33 (9.30 MB/s) - ‘data/full_dataset/goemotions_1.csv’ saved [14174600/14174600]



In [ ]:
df2=pd.read_csv("data/full_dataset/goemotions_1.csv")

df2 = df2.drop(['author','example_very_unclear','id', 'subreddit','link_id','parent_id','created_utc','rater_id'], axis=1)

In [ ]:
def classify(row):
  if row.amusement == 1 or row.curiosity == 1 or row.excitement == 1 or row.pride == 1 or row.surprise == 1:
    return 'surprise'
  if row.anger == 1 or row.annoyance == 1:
    return 'anger'
  if row.caring == 1 or row.gratitude == 1 or row.joy == 1  or row.relief == 1:
    return 'joy'
  if row.admiration == 1 or row.love==1 or row.desire==1:
    return 'love'
  if row.disappointment == 1 or row.disapproval == 1 or row.disgust == 1 or row.sadness == 1 or row.grief==1:
    return 'sadness'
  if row.fear==1 or row.nervousness == 1:
    return 'fear'

In [ ]:
LABEL=[]
for index, row in df2.iterrows():
    LABEL.append(classify(row))

df2['LABEL']=LABEL

In [ ]:
df2=df2.drop(["admiration","amusement","anger","annoyance","approval","caring","confusion","curiosity","desire","disappointment","disapproval","disgust","embarrassment","excitement","fear","gratitude","grief","joy","love","nervousness","optimism","pride","realization","relief","remorse","sadness","surprise","neutral"],axis=1)

In [ ]:
df2.describe()

,text,LABEL
count,70000,40287
unique,43182,6
top,Thank you.,surprise
freq,14,10081


In [ ]:
df2=df2.dropna()


In [ ]:
df2.describe()

,text,LABEL
count,40287,40287
unique,28882,6
top,Thank you.,surprise
freq,14,10081


In [ ]:
df_new=df_full.rename(columns={'Input':'text','Sentiment':'LABEL'})
df_new

,text,LABEL
0,i didnt feel humiliated,sadness
1,i can go from feeling so hopeless to so damned...,sadness
2,im grabbing a minute to post i feel greedy wrong,anger
3,i am ever feeling nostalgic about the fireplac...,love
4,i am feeling grouchy,anger
...,...,...
1995,im having ssa examination tomorrow in the morn...,sadness
1996,i constantly worry about their fight against n...,joy
1997,i feel its important to share this info for th...,joy
1998,i truly feel that if you are passionate enough...,joy


In [ ]:
df_final=pd.concat([df2,df_new])

In [ ]:
df_final.describe()

,text,LABEL
count,60287,60287
unique,48830,6
top,Thank you.,joy
freq,14,14343


In [ ]:
df_final = df_final.sample(frac=0.3, random_state=42)

In [ ]:
df_final.shape

(18086, 2)

In [ ]:

label2id = { "sadness":0,
"anger":1,
"love":2,
"surprise":3,
"fear":4,
"joy":5
}

id2label = {v: k for k, v in label2id.items()}

In [ ]:
def classify(row):
  
  
  if row.LABEL=="sadness":
    return 0

  if row.LABEL=="anger":
    return 1

  if row.LABEL=="love":
    return 2

  if row.LABEL=="surprise":
    return 3

  if row.LABEL=="fear":
    return 4

  if row.LABEL=="joy":
    return 5

In [ ]:
label=[]
for index, row in df_final.iterrows():
    label.append(classify(row))



In [ ]:

df_final["label"]=label
df_final= df_final.drop(["LABEL"], axis=1)
df_final.head()

,text,label
17026,"Incredible work and very unique, good for you",3
63409,“That’s up to you” Clearly it is not up to you...,1
15771,i beg and crave a particular something that im...,0
26275,Funny they decided to show a bunch of T2 image...,3
3962,"Ah, the ol' plurality of majesty.",2


In [ ]:
df_final.shape

(18086, 2)

In [ ]:
!pip install text-hammer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 KB 7.8 MB/s eta 0:00:00
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [ ]:
import text_hammer as th



from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas()

def clean_text(df,col_name):
    column = col_name
    df[column] = df[column].progress_apply(lambda x:str(x).lower())
    df[column] = df[column].progress_apply(lambda x: th.cont_exp(x)) 
    df[column] = df[column].progress_apply(lambda x: th.remove_emails(x))
    df[column] = df[column].progress_apply(lambda x: th.remove_html_tags(x))

    df[column] = df[column].progress_apply(lambda x: th.remove_special_chars(x))
    df[column] = df[column].progress_apply(lambda x: th.remove_accented_chars(x))
    return(df)

In [ ]:
df_final = clean_text(df_final,"text")

  0%|          | 0/18086 [00:00<?, ?it/s]

  0%|          | 0/18086 [00:00<?, ?it/s]

  0%|          | 0/18086 [00:00<?, ?it/s]

  0%|          | 0/18086 [00:00<?, ?it/s]

  0%|          | 0/18086 [00:00<?, ?it/s]

  0%|          | 0/18086 [00:00<?, ?it/s]

In [ ]:
df_final.label.unique()

array([3, 1, 0, 2, 5, 4])

In [ ]:
from transformers import BertTokenizer, TFBertModel, BertConfig, TFBertForSequenceClassification
tokenizer= BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case= True)

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
texts = df_final.text.values
labels= df_final.label.values
from sklearn.model_selection import train_test_split
train_input, test_input, train_label, val_label = train_test_split(texts, labels)
max_len =0
for tex in texts:
  max_len=max(max_len, len(tex))

print(max_len)

703


In [ ]:
import tensorflow as tf

In [ ]:
def mask_inputs_for_bert(texts, max_len):
  input_ids=[]
  attention_masks=[]
  i=0
  for tex in texts: 
    if(i<3):
      print("Text", tex)  

    encoded_dict =tokenizer.encode_plus(tex, add_special_tokens=True, truncation=True,
                                        max_length= max_len, 
                                        pad_to_max_length=True,
                                        return_attention_mask=True
                                       )
    if(i<3):  
      print("dict", encoded_dict["input_ids"])
    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])

    i=i+1

  input_ids=tf.convert_to_tensor(input_ids)
  attention_masks= tf.convert_to_tensor(attention_masks)

  return input_ids, attention_masks


In [ ]:
train_inp, train_mask= mask_inputs_for_bert(train_input, max_len)
val_inp, val_mask= mask_inputs_for_bert(test_input, max_len)
train_label= tf.convert_to_tensor(train_label)
val_label= tf.convert_to_tensor(val_label)

Text that is absurd people commit crimes blacked out on drugsalcohol all the time also they could just claim they do not remember it either way
dict [101, 2008, 2003, 18691, 2111, 10797, 6997, 2304, 2098, 2041, 2006, 5850, 2389, 3597, 14854, 2035, 1996, 2051, 2036, 2027, 2071, 2074, 4366, 2027, 2079, 2025, 3342, 2009, 2593, 2126, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2336: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Text i feel im forever alone
dict [101, 1045, 2514, 10047, 5091, 2894, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [ ]:
log_dir= '/content/drive/MyDrive'
#log_dir="logs"
model_save_path='/content/emoTer/bert_model.h5'

In [ ]:
import keras

model_checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath= model_save_path,\
                                               ),\
            keras.callbacks.TensorBoard(log_dir=log_dir)

In [ ]:
bert_model= TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=6)

Downloading:   0%|          | 0.00/536M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
callbacks= [model_checkpoint]

print("Summary of model\n", bert_model.summary())
loss= tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric= tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
optimizer=tf.keras.optimizers.Adam(learning_rate=1e-6)

Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  109482240 
                                                                 
 dropout_37 (Dropout)        multiple                  0         
                                                                 
 classifier (Dense)          multiple                  4614      
                                                                 
Total params: 109,486,854
Trainable params: 109,486,854
Non-trainable params: 0
_________________________________________________________________
Summary of model
 None


In [ ]:
bert_model.compile(loss=loss, optimizer= optimizer, metrics=[metric])

In [ ]:
history= bert_model.fit([train_inp, train_mask],\
                        train_label,\
                        batch_size=2,\
                        epochs=4,\
                        validation_data=([val_inp, val_mask], val_label))

Epoch 1/4
6782/6782 [==============================] - 2959s 436ms/step - loss: 1.5377 - accuracy: 0.3910 - val_loss: 1.2691 - val_accuracy: 0.5228
Epoch 2/4
6782/6782 [==============================] - 2968s 438ms/step - loss: 1.1624 - accuracy: 0.5680 - val_loss: 1.0840 - val_accuracy: 0.6165
Epoch 3/4
6782/6782 [==============================] - 2916s 430ms/step - loss: 0.9898 - accuracy: 0.6474 - val_loss: 1.0022 - val_accuracy: 0.6468
Epoch 4/4
6782/6782 [==============================] - 2916s 430ms/step - loss: 0.8721 - accuracy: 0.6886 - val_loss: 0.9393 - val_accuracy: 0.6623
